In [ ]:
#imports
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

TODO if time: vectorization

convolution, edge detection (sobel), corner detection, ransac - done in assignments

In [ ]:
#hough transform

In [ ]:
def compute_homography(src, dst):
    # Compute the homography matrix from source to destination points.
    A = []
    for i in range(len(src)):
        x_src, y_src = src[i][0], src[i][1]
        x_dst, y_dst = dst[i][0], dst[i][1]
        A.append([-x_src, -y_src, -1, 0, 0, 0, x_src * x_dst, y_src * x_dst, x_dst])
        A.append([0, 0, 0, -x_src, -y_src, -1, x_src * y_dst, y_src * y_dst, y_dst])
        print(A)

    A = np.array(A)
    print('hey', A)
    U, S, V = np.linalg.svd(A)
    print('lemons', A)
    print(f'u = {U}, \n s = {S},\n v = {V}', )
    H = V[-1].reshape(3, 3)
    print('make lemonades', A)

    return H / H[-1, -1]  #normalizing

src = np.array([[100, 100], [200, 100], [200, 200], [100, 200]])
dst = np.array([[120, 120], [240, 100], [220, 250], [100, 200]])

H = compute_homography(src, dst)
print("Homography matrix:\n", H)

[[-100, -100, -1, 0, 0, 0, 12000, 12000, 120], [0, 0, 0, -100, -100, -1, 12000, 12000, 120]]
[[-100, -100, -1, 0, 0, 0, 12000, 12000, 120], [0, 0, 0, -100, -100, -1, 12000, 12000, 120], [-200, -100, -1, 0, 0, 0, 48000, 24000, 240], [0, 0, 0, -200, -100, -1, 20000, 10000, 100]]
[[-100, -100, -1, 0, 0, 0, 12000, 12000, 120], [0, 0, 0, -100, -100, -1, 12000, 12000, 120], [-200, -100, -1, 0, 0, 0, 48000, 24000, 240], [0, 0, 0, -200, -100, -1, 20000, 10000, 100], [-200, -200, -1, 0, 0, 0, 44000, 44000, 220], [0, 0, 0, -200, -200, -1, 50000, 50000, 250]]
[[-100, -100, -1, 0, 0, 0, 12000, 12000, 120], [0, 0, 0, -100, -100, -1, 12000, 12000, 120], [-200, -100, -1, 0, 0, 0, 48000, 24000, 240], [0, 0, 0, -200, -100, -1, 20000, 10000, 100], [-200, -200, -1, 0, 0, 0, 44000, 44000, 220], [0, 0, 0, -200, -200, -1, 50000, 50000, 250], [-100, -200, -1, 0, 0, 0, 10000, 20000, 100], [0, 0, 0, -100, -200, -1, 20000, 40000, 200]]
hey [[ -100  -100    -1     0     0     0 12000 12000   120]
 [    0     0  

In [ ]:
#normalizing 2d points in homogenous coordinates
"""
step 1: compute centroid
step 2: translate points (place centroid at origin) - points[:, :2] - centroid
step 3: compute avg distance from origin
step 4: compute scaling factor (w/ mean distance sqrt(2))
step 5: create the normalization matrix
step 6: apply the normalization matrix to the points
 """

def normalize(points):
    centroid = np.mean(points[:, :2], axis=0)
    translated_points = points[:, :2] - centroid
    avg_distance = np.mean(np.sqrt(np.sum(translated_points**2, axis=1)))
    scaling_factor = np.sqrt(2) / avg_distance

    T = np.array([[scaling_factor, 0, -scaling_factor * centroid[0]],
                  [0, scaling_factor, -scaling_factor * centroid[1]],
                  [0, 0, 1]])

    nm_pts = (T @ points.T).T
    return nm_pts, T


points = np.array([[100, 100, 1],
                   [200, 100, 1],
                   [200, 200, 1],
                   [100, 200, 1]])

nm_pts, T = normalize(points)

print("Normalized Points:\n", nm_pts)
print("Normalization Matrix T:\n", T)

Normalized Points:
 [[-1. -1.  1.]
 [ 1. -1.  1.]
 [ 1.  1.  1.]
 [-1.  1.  1.]]
Normalization Matrix T:
 [[ 0.02  0.   -3.  ]
 [ 0.    0.02 -3.  ]
 [ 0.    0.    1.  ]]


fixed some stuff w/ gpt from here:

In [ ]:
#applying homography - get points and apply

def homography(points, H):
    points_homogeneous = np.column_stack((points, np.ones(points.shape[0])))
    transformed_homogeneous = points_homogeneous @ H.T
    transformed_points = transformed_homogeneous[:, :2] / transformed_homogeneous[:, 2, np.newaxis]
    return transformed_points

points = np.array([[100, 100], [200, 100], [200, 200], [100, 200]])
H = np.array([[1.5, 0, 50], [0, 1.5, 30], [0, 0, 1]])

transformed_points = homography(points, H)
print("Original Points:\n", points)
print("Transformed Points:\n", transformed_points)

In [ ]:
#project 3d points onto a 2d plane using the camera projection matrix P
def threedto2d(P, points_3d):
    points_2d = np.dot(np.column_stack((points_3d, np.ones(points_3d.shape[0]))), P.T)
    return (points_2d[:, :2].T / points_2d[:, 2]).T

points_3d = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
P = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 1]])

points_2d = threedto2d(P, points_3d)
print("Projected 2D points:\n", points_2d)

Projected 2D points:
 [[0.25       0.5       ]
 [0.57142857 0.71428571]
 [0.7        0.8       ]]


In [ ]:
# affine:

# fitting affine transformation two 6 sets of points
import numpy as np

A = np.empty((0,6),int)
B = np.array([])
X = np.array([[100, 100], [200, 100], [200, 200]])
X_transformed = np.array([[120, 120], [240, 100], [220, 250]])

for i in range(len(X)):
    A = np.append(A, np.array([[X[i][0],X[i][1],0,0,1,0]]),axis=0)
    A = np.append(A,np.array([[0,0,X[i][0],X[i][1],0,1]]),axis=0)
    B = np.append(B,[X_transformed[i][0],X_transformed[i][1]])

print(A,B.T)

print("Compute affine transformation matrix:")
M = np.linalg.solve(A,B.T)
print(M)

print(np.allclose(np.dot(A, M), B.T))

In [ ]:
#compute the fundamental matrix (??? maybe - got an error last time)
import numpy as np

def compute_fundamental_matrix(pts1, pts2):
    def normalize(points):
        mean = np.mean(points, axis=0)
        std_dev = np.std(points)
        scale = np.sqrt(2) / std_dev
        T = np.array([[scale, 0, -scale * mean[0]],
                      [0, scale, -scale * mean[1]],
                      [0, 0, 1]])
        return np.dot(T, points.T).T, T

    pts1_normalized, T1 = normalize(pts1)
    pts2_normalized, T2 = normalize(pts2)

    A = np.array([[x1 * x2, x1 * y2, x1, y1 * x2, y1 * y2, y1, x2, y2, 1]
                  for (x1, y1), (x2, y2) in zip(pts1_normalized, pts2_normalized)])

    _, _, V = np.linalg.svd(A)
    F = V[-1].reshape(3, 3)

    U, S, Vt = np.linalg.svd(F)
    S[2] = 0
    F = np.dot(U, np.dot(np.diag(S), Vt))

    F = T2.T @ F @ T1

    return F / F[-1, -1]

pts1 = np.array([[100, 100, 1], [200, 100, 1], [200, 200, 1], [100, 200, 1],
                 [150, 150, 1], [250, 150, 1], [250, 250, 1], [150, 250, 1]])
pts2 = np.array([[120, 120, 1], [240, 100, 1], [220, 250, 1], [100, 200, 1],
                 [170, 170, 1], [270, 150, 1], [270, 270, 1], [170, 270, 1]])

F = compute_fundamental_matrix(pts1, pts2)
print("Fundamental Matrix:\n", F)

In [ ]:
#compute epipolar lines for the given fundamental matrix
import numpy as np

def compute_epipolar_lines(pts, F):
    return np.dot(F, pts.T).T

lines = compute_epipolar_lines(pts1, F)
print("Epipolar Lines:\n", lines)

In [ ]:
#look into moreeeeee
import matplotlib.pyplot as plt

def epipolar_lines(image_shape, lines, points, ax):
    h, w = image_shape
    for line, pt in zip(lines, points):
        a, b, c = line
        x0, x1 = 0, w
        y0 = -(c + a * x0) / b
        y1 = -(c + a * x1) / b
        ax.plot([x0, x1], [y0, y1], color='red')
        ax.plot(pt[0], pt[1], 'bo')

fig, ax = plt.subplots(figsize=(8, 6))
image_shape = (400, 400)
ax.set_xlim(0, image_shape[1])
ax.set_ylim(image_shape[0], 0)

epipolar_lines(image_shape, lines, pts1, ax)

ax.set_title("Epipolar Lines Visualization")
ax.set_xlabel("X-axis")
ax.set_ylabel("Y-axis")
plt.grid()
plt.show()

j gpt-ed implementations:

In [ ]:
import numpy as np

def median_filter(image, kernel_size=3):
    # Pad the image to handle borders
    pad_size = kernel_size // 2
    padded_image = np.pad(image, pad_size, mode='edge')

    # Prepare an output image
    filtered_image = np.zeros_like(image)

    # Iterate over every pixel in the image
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            # Extract the region of interest
            region = padded_image[i:i + kernel_size, j:j + kernel_size]
            # Compute the median of the region
            filtered_image[i, j] = np.median(region)

    return filtered_image

# Example usage
if __name__ == "__main__":
    # Create a sample grayscale image (2D array)
    sample_image = np.array([[100, 200, 100],
                              [200, 50, 200],
                              [100, 200, 100]], dtype=np.uint8)

    # Apply the median filter
    filtered_image = median_filter(sample_image, kernel_size=3)

    print("Original Image:")
    print(sample_image)
    print("Filtered Image:")
    print(filtered_image)

In [ ]:
#applying a gaussian filter
import numpy as np

def gaussian_kernel(size, sigma=1):
    """Generate a Gaussian kernel."""
    kernel = np.fromfunction(
        lambda x, y: (1 / (2 * np.pi * sigma**2)) *
                      np.exp(-((x - (size - 1) / 2)**2 + (y - (size - 1) / 2)**2) / (2 * sigma**2)),
        (size, size)
    )
    return kernel / np.sum(kernel)

def gaussian_filter(image, kernel_size=3, sigma=1):
    """Apply a Gaussian filter to an image."""
    # Create Gaussian kernel
    kernel = gaussian_kernel(kernel_size, sigma)

    # Pad the image to handle borders
    pad_size = kernel_size // 2
    padded_image = np.pad(image, pad_size, mode='edge')

    # Prepare an output image
    filtered_image = np.zeros_like(image)

    # Iterate over every pixel in the image
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            # Extract the region of interest
            region = padded_image[i:i + kernel_size, j:j + kernel_size]
            # Apply the kernel (convolution)
            filtered_image[i, j] = np.sum(region * kernel)

    return filtered_image

# Example usage
if __name__ == "__main__":
    # Create a sample grayscale image (2D array)
    sample_image = np.array([[100, 200, 100],
                              [200, 50, 200],
                              [100, 200, 100]], dtype=np.float32)

    # Apply the Gaussian filter
    filtered_image = gaussian_filter(sample_image, kernel_size=3, sigma=1)

    print("Original Image:")
    print(sample_image)
    print("Filtered Image:")
    print(filtered_image)

In [ ]:
#least squares
import numpy as np
import matplotlib.pyplot as plt

# Example data: x (independent variable) and y (dependent variable)
x = np.array([1, 2, 3, 4, 5])
y = np.array([2, 3, 5, 7, 11])

# Number of data points
n = len(x)

# Calculate the slope (m) and intercept (b) using least squares
#j need this
m = (n * np.sum(x * y) - np.sum(x) * np.sum(y)) / (n * np.sum(x**2) - (np.sum(x))**2)
b = (np.sum(y) - m * np.sum(x)) / n

# Print the results
print(f"Slope (m): {m:.2f}, Intercept (b): {b:.2f}")

# Calculate predicted values (y_hat)
y_hat = m * x + b

# Plotting the data points and the fitted line
plt.scatter(x, y, color='blue', label='Data points')
plt.plot(x, y_hat, color='red', label='Fitted line')
plt.xlabel('Independent Variable (x)')
plt.ylabel('Dependent Variable (y)')
plt.title('Least Squares Linear Regression')
plt.legend()
plt.grid()
plt.show()